In [1]:
import json
import os
import pandas as pd
import pyarrow as pa
import pyarrow.dataset as ds
import torch
import wandb

from datasets import Dataset
from peft import LoraConfig
from trl import ModelConfig, SFTConfig, SFTTrainer
from transformers import AutoModelForCausalLM, TrainingArguments

MODEL_NAME = 'AnatoliiPotapov/T-lite-instruct-0.1'

wandb.finish()
os.environ['WANDB_DISABLED'] = 'true'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
texts = [
    {
        "messages": [
            {"role": "user", "content": "Каковы текущие котировки акций компании Apple?"},
            {"role": "bot", "content": "Текущие котировки акций Apple составляют $173.21."},
        ]
    },
    {
        "messages": [
            {"role": "user", "content": "Какие экономические отчеты ожидаются на этой неделе?"},
            {"role": "bot", "content": "На этой неделе ожидаются отчеты по безработице и ВВП США."},
        ]
    },
]

texts = [json.dumps(text) for text in texts]

df = pd.DataFrame(
    {
        'text': texts,
        'label': [0 for _ in range(len(texts))]
    }
)
dataset = ds.dataset(pa.Table.from_pandas(df).to_batches())

hg_dataset = Dataset(pa.Table.from_pandas(df))

In [ ]:
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)

In [ ]:
# Задаем настройки модели и обучения 
# в конфигурацию модели передаем способ обучения flash_attention_2 - алгоритм,
# позволяющий ускорить процесс глубокого обучения в части механизма внимания
model_config = ModelConfig(
    model_name_or_path=MODEL_NAME,
    attn_implementation='flash_attention_2',
)

sft_config = TrainingArguments(
    output_dir='test_trainer',
)

In [ ]:
# Настраиваем LoRA 
# (для предварительно обученной матрицы весов мы представляем её обновление двумя меньшими матрицами, 
# полученными путем низкоранговой аппроксимации)
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM',
)

In [ ]:
# Готовим и запускаем основной процесс дообучения модели
trainer = SFTTrainer(
    model,
    train_dataset=hg_dataset,
    args=sft_config,
    dataset_text_field='text',
    packing=False,
    peft_config=peft_config,
    max_seq_length=256
)

trainer.train()